In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive





# 학습된 Fasttext 적용 해서 X_train, y_train, X_test 만들기




In [ ]:
import pandas as pd
train_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/train_preprocesed.csv")
test_preprocessed = pd.read_csv("/content/MyDrive/MyDrive/통계청/test_preprocessed.csv")

In [ ]:
!pip install jamo

In [ ]:
from jamo import h2j, j2hcj
#자모단위 자르는 함수
def word2jamo(sen):
    return j2hcj(h2j(sen))
train_preprocessed['jamo'] = train_preprocessed['document'].apply(word2jamo)
test_preprocessed['jamo'] = test_preprocessed['document'].apply(word2jamo)

In [ ]:
!pip install fasttext

     |████████████████████████████████| 68 kB 3.1 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3132201 sha256=f10cc4c64d74072c6dbdbc47f2c2056522a1ac3c9dc71db8434fb450225d2208
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [ ]:
import fasttext

fasttext_embedding_학습_ver0.1에서 학습한 fasttext 임베딩 모델 불러오기

In [ ]:
model = fasttext.load_model('/content/MyDrive/MyDrive/통계청/embeddingmodels/fasttext_ver0.1.bin')

In [ ]:
vec_list_train = []
for i in train_preprocessed['jamo']:
    vec = model.get_sentence_vector(i)
    vec_list_train.append(vec)

vec_list_test = []
for i in test_preprocessed['jamo']:
    vec = model.get_sentence_vector(i)
    vec_list_test.append(vec)

label 개수가 230개미만인것 없애기

In [ ]:
train_preprocessed['emb_vec'] = vec_list_train
test_preprocessed['emb_vec'] = vec_list_test

In [ ]:
countdf = train_preprocessed.groupby('label').count().sort_values(by='document')
over230labels = countdf[countdf['document']>230].index.to_list()
train_preprocessed = train_preprocessed[train_preprocessed['label'].isin(over230labels)]

train_processed 에서 임베딩벡터와 label을 각각 X_train,y_train 로 뽑아내서 저장, test_preprecessed의 임베딩벡터를 X_test로 저장

In [ ]:
X_train = np.array(train_preprocessed['emb_vec'].to_list())
y_train = np.array(train_preprocessed['label'].to_list())
X_test = np.array(test_preprocessed['emb_vec'].to_list())


In [ ]:
X_train.shape

(995442, 100)

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_train_ver0.2.npy', X_train)

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_test_ver0.2.npy', X_test)

In [ ]:
np.save('/content/MyDrive/MyDrive/통계청/embedded_numpy/y_train_ver0.2.npy', y_train)

# 딥러닝 모델 구축 시작

In [ ]:
import numpy as np
X_train = np.load('/content/MyDrive/MyDrive/통계청/embedded_numpy/X_train_ver0.2.npy',allow_pickle=True)
y_train = np.load('/content/MyDrive/MyDrive/통계청/embedded_numpy/y_train_ver0.2.npy',allow_pickle=True)

In [ ]:
X_train

array([[ 0.06743513,  0.02341077, -0.07920723, ...,  0.15397546,
        -0.00648655, -0.10477634],
       [ 0.04290798,  0.09327256, -0.06156277, ...,  0.09845879,
         0.03878662, -0.03403267],
       [ 0.0573958 ,  0.0902684 , -0.01492267, ...,  0.02671049,
         0.07588854,  0.01236362],
       ...,
       [ 0.08015418,  0.05722065, -0.02062835, ...,  0.06784131,
         0.03634049,  0.00606036],
       [ 0.02017494, -0.01451707, -0.0489815 , ...,  0.05661954,
        -0.01008923,  0.00932857],
       [ 0.06786437,  0.03006427,  0.02305304, ...,  0.11821701,
        -0.02131855,  0.0513864 ]], dtype=float32)

In [ ]:
y_train

array(['S95952', 'G47472', 'G46467', ..., 'G47474', 'P85856', 'I56561'],
      dtype='<U6')

In [ ]:
import tensorflow as tf
import pandas as pd

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping


In [ ]:
y_train

array(['S95952', 'G47472', 'G46467', ..., 'G47474', 'P85856', 'I56561'],
      dtype='<U6')

X_train과 y_train을 trainset,validation set으로 분할

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encoder.fit(y_train)
y_train_digit = encoder.transform(y_train)
x_train, x_val, y_train, y_val = train_test_split(X_train,y_train_digit, 
                                                    test_size=0.2,  
                                                    random_state=1004)



In [ ]:
len(set(y_train))

168

In [ ]:
y_train.shape

(796353,)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation


model1 = Sequential(name = 'model1')
model1.add(Dense(100, activation='relu', input_dim=(100)))
model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1024, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(168, activation='softmax'))
model1.summary()


Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               10100     
                                                                 
 dense_25 (Dense)            (None, 1024)              103424    
                                                                 
 dropout_12 (Dropout)        (None, 1024)              0         
                                                                 
 dense_26 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_13 (Dropout)        (None, 1024)              0         
                                                                 
 dense_27 (Dense)            (None, 168)               172200    
                                                                 
Total params: 1,335,324
Trainable params: 1,335,324
Non-trai

In [ ]:
model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import keras

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2,restore_best_weights=True)
history = model1.fit(x_train,y_train,
                    epochs=100,
                    batch_size=100,
                    validation_data=(x_val,y_val),
                    callbacks=early_stop)

Epoch 1/100
7964/7964 [==============================] - 32s 4ms/step - loss: 0.3003 - accuracy: 0.9129 - val_loss: 0.3342 - val_accuracy: 0.9084
Epoch 2/100
7964/7964 [==============================] - 28s 4ms/step - loss: 0.2963 - accuracy: 0.9138 - val_loss: 0.3352 - val_accuracy: 0.9081
Epoch 3/100
7964/7964 [==============================] - 28s 3ms/step - loss: 0.2931 - accuracy: 0.9146 - val_loss: 0.3370 - val_accuracy: 0.9091


In [ ]:
model1.save('/content/MyDrive/MyDrive/통계청/deeplearningmodel/model_ver0.1.h5')

In [ ]:
y_test_encoded = model1.predict(X_test)

In [99]:
answer = encoder.inverse_transform(y_test_encoded.argmax(axis=1))

In [100]:
ansdf = pd.DataFrame(columns = ['label','digit_1','digit_2','digit_3'])
ansdf['label'] = answer
ansdf['digit_1'] = ansdf['label'].str[:1]
ansdf['digit_2'] = ansdf['label'].str[1:3]
ansdf['digit_3'] = ansdf['label'].str[3:]
ansdf[['digit_1','digit_2','digit_3']].to_csv('/content/MyDrive/MyDrive/통계청/deeplearning_model_Ver0.1.csv',encoding='utf-8')